In [1]:
import os, subprocess
import json
import uproot3
import awkward as ak
import numpy as np
from coffea import processor, util, hist

from plotter import *

In [2]:
lumis = {}
lumis['2016'] = 35.9
lumis['2017'] = 41.5
lumis['2018'] = 59.9

with open('xsec.json') as f:
  xs = json.load(f)

with open('pmap.json') as f:
  pmap = json.load(f)

In [3]:
deta_cut = 3.5
mjj_cut = 500

deta_cut_mucr = 0
mjj_cut_mucr = 0

In [4]:
year = '2017'
nfiles = len(subprocess.getoutput("ls infiles-split/"+year+"*.json").split())
outsum = processor.dict_accumulator()

In [5]:
# Load all files
for n in range(1,nfiles+1):

    with open('infiles-split/'+year+'_'+str(n)+'.json') as f:
      infiles = json.load(f)
    
    filename = '/myeosdir/vbf-category/outfiles/'+year+'_'+str(n)+'.coffea'
    #filename = 'outfiles/'+year+'_'+str(n)+'.coffea'
    if os.path.isfile(filename):
        out = util.load(filename)
        outsum.add(out)
    else:
        print(n,infiles.keys())
        #print("File " + filename + " is missing")
        
scale_lumi = {k: xs[k] * 1000 *lumis[year] / w for k, w in outsum['sumw'].items()}
outsum['templates-vbf'].scale(scale_lumi, 'dataset')
outsum['templates-vbf-2'].scale(scale_lumi, 'dataset')

templates_vbf = outsum['templates-vbf'].group('dataset', hist.Cat('process', 'Process'), pmap).integrate('region','signal')
templates_vbf = templates_vbf.integrate('deta',int_range=slice(deta_cut,7.0)).integrate('mjj',int_range=slice(mjj_cut,4000))
templates_vbf.sum('pt1','msd1').integrate('ddb1',int_range=slice(0.89,1)).values()

del outsum
print('ok')

ValueError: Cannot add this histogram with histogram <Hist (dataset,region,systematic,pt1,msd1,ddb1,deta,mjj) instance at 0x7fe2c1067a90> of dissimilar dimensions

In [ ]:
systematics = ['nominal',
               'jet_triggerUp','jet_triggerDown',
               'btagWeightUp','btagWeightDown','btagEffStatUp','btagEffStatDown',
               'UESUp','UESDown','JESUp','JESDown','JERUp','JERDown',
               'PS_weightUp','PS_weightDown'
              ]

In [ ]:
split = [500,550,600,675,800]

for x in split:
    for y in split:
        if x >= y:
            continue
        
        ptbins = [450, x, y, 1200]
        os.system('rm '+year+'/'+str(x)+'_'+str(y)+'/3pt-signalregion.root')
        fout = uproot3.create(year+'/3pt-signalregion.root')
        for i,b in enumerate(ptbins[:-1]):
            for p in pmap.keys(): 
                if p == "ttH" and year == '2016':
                    continue
            print(p)
            if "data" in p:
                s = "nominal"
                h = templates_vbf.integrate('systematic',s).integrate('pt1',int_range=slice(ptbins[i],ptbins[i+1])).integrate('ddb1',int_range=slice(0.89,1)).integrate('process',p)
                fout["pass_pt"+str(i+1)+"_"+p+"_"+s] = hist.export1d(h)
                h = templates_vbf.integrate('systematic',s).integrate('pt1',int_range=slice(ptbins[i],ptbins[i+1])).integrate('ddb1',int_range=slice(0,0.89)).integrate('process',p)
                fout["fail_pt"+str(i+1)+"_"+p+"_"+s] = hist.export1d(h)
            else:
                for s in systematics:
                    h = templates_vbf.integrate('systematic',s).integrate('pt1',int_range=slice(ptbins[i],ptbins[i+1])).integrate('ddb1',int_range=slice(0.89,1)).integrate('process',p)
                    fout["pass_pt"+str(i+1)+"_"+p+"_"+s] = hist.export1d(h)
                    h = templates_vbf.integrate('systematic',s).integrate('pt1',int_range=slice(ptbins[i],ptbins[i+1])).integrate('ddb1',int_range=slice(0,0.89)).integrate('process',p)
                    fout["fail_pt"+str(i+1)+"_"+p+"_"+s] = hist.export1d(h)

        fout.close()

In [ ]:
os.system('rm '+year+'/muonCR.root')
fout = uproot3.create(year+'/muonCR.root')
for p in pmap.keys():  
    if p == 'ttH' and year == '2016':
        continue
    print(p)
    if "data" in p:
        s = "nominal"
        h = templates_vbf_mucr.integrate('systematic',s).sum('pt1').integrate('ddb1',int_range=slice(0.89,1)).integrate('process',p)
        fout["pass_"+p+"_"+s] = hist.export1d(h)
        h = templates_vbf_mucr.integrate('systematic',s).sum('pt1').integrate('ddb1',int_range=slice(0,0.89)).integrate('process',p)
        fout["fail_"+p+"_"+s] = hist.export1d(h)
    else:
        for s in systematics:
            h = templates_vbf_mucr.integrate('systematic',s).sum('pt1').integrate('ddb1',int_range=slice(0.89,1)).integrate('process',p)
            fout["pass_"+p+"_"+s] = hist.export1d(h)
            h = templates_vbf_mucr.integrate('systematic',s).sum('pt1').integrate('ddb1',int_range=slice(0,0.89)).integrate('process',p)
            fout["fail_"+p+"_"+s] = hist.export1d(h)